In [1]:
%load_ext autoreload
%autoreload 2

from pathlib import Path
from multiplex_pipeline.utils.config_loaders import load_analysis_settings
from multiplex_pipeline.utils.globus_utils import GlobusConfig, create_globus_tc
from multiplex_pipeline.utils.file_utils import globus_dir_exists, GlobusPathConverter
from multiplex_pipeline.core_cutting.file_io import GlobusFileStrategy

c:\Users\KMK280\AppData\Local\miniconda3\envs\sdata-env\Lib\site-packages\dask\dataframe\__init__.py:31: FutureWarning: The legacy Dask DataFrame implementation is deprecated and will be removed in a future version. Set the configuration option `dataframe.query-planning` to `True` or None to enable the new Dask Dataframe implementation and silence this warning.
  warnings.warn(
c:\Users\KMK280\AppData\Local\miniconda3\envs\sdata-env\Lib\site-packages\xarray_schema\__init__.py:1: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import DistributionNotFound, get_distribution


In [2]:
# experiment settings file path
settings_path = r'C:\sdata_NRR-W\analysis_settings_NRR-W.yaml'

# globus config
config_path = Path(r'D:\globus_config\globus_config.yaml')

In [3]:
# load analysis configuration
settings = load_analysis_settings(settings_path)
settings

AnalysisConfig(general=GeneralSettings(image_dir='R:/CellDive/NRR-W/NRR-W_Final', analysis_name='NRR-W_Analysis', local_analysis_dir='C:/sdata_NRR-W', remote_analysis_dir='/ix1/kkedziora', log_dir=None), core_detection=CoreDetectionSettings(detection_image='NRR-W_1.0.4_R000_DAPI__FINAL_F.ome.tif', core_info_file_path=None, im_level=6, min_area=2000, max_area=10000, min_iou=0.8, min_st=0.9, min_int=15, frame=4), core_cutting=CoreCuttingSettings(cores_dir_tif=None, cores_dir_output=None, include_channels=None, exclude_channels=None, use_markers=None, ignore_markers=['Antibody1'], margin=0, mask_value=0, transfer_cleanup_enabled=True, core_cleanup_enabled=True), additional_elements=[NormalizeStep(category='image_transformer', type='normalize', input='DAPI', output='DAPI_norm', keep=False, parameters=Params(low=1.0, high=99.8)), NormalizeStep(category='image_transformer', type='normalize', input='ECad', output='ECad_norm', keep=False, parameters=Params(low=1.0, high=99.8)), InstansegStep(c

In [4]:
# get Globus settings
gc = GlobusConfig.from_config_files(config_path, from_collection = 'cbi_collection_id', to_collection = 'crcd_collection_id')
tc = create_globus_tc(gc.client_id, gc.transfer_tokens)

In [5]:
# check if the destination analysis directory exists 
remote_analysis_dir = settings.general.remote_analysis_dir + '/' + settings.general.analysis_name

globus_dir_exists(tc, gc.destination_collection_id, remote_analysis_dir)

True

In [6]:
# build a transfer map

# if transfering from Windows with multiple drives exposed in Globus use "multi_drive"
# if transfering from Windows with a single drive exposed in Globus 
# or if transfering from CIFs drive use "single_drive"
conv = GlobusPathConverter(layout="multi_drive")

local_settings_path = conv.windows_to_globus(settings_path)
remote_settings_path = remote_analysis_dir + '/' + Path(settings_path).name
    
local_cores_info_path = conv.windows_to_globus(settings.core_info_file_path.with_suffix('.pkl'))
remote_cores_info_path = remote_analysis_dir + '/' + Path(local_cores_info_path).name

remote_cores_info_path = remote_analysis_dir + '/' + Path(local_cores_info_path).name

transfer_map = {}
transfer_map['settings'] = (local_settings_path, remote_settings_path)
transfer_map['core_definitions'] = (local_cores_info_path, remote_cores_info_path)
transfer_map

{'settings': ('/C/sdata_NRR-W/analysis_settings_NRR-W.yaml',
  '/ix1/kkedziora/NRR-W_Analysis/analysis_settings_NRR-W.yaml'),
 'core_definitions': ('/C/sdata_NRR-W/NRR-W_Analysis/cores.pkl',
  '/ix1/kkedziora/NRR-W_Analysis/cores.pkl')}

In [7]:
# submit transfer
transfer = GlobusFileStrategy(tc=tc, transfer_map=transfer_map, gc=gc, cleanup_enabled = False)

2025-11-06 13:45:37.149 | INFO     | multiplex_pipeline.core_cutting.file_io:submit_all_transfers:91 - Submitted transfer for settings to /ix1/kkedziora/NRR-W_Analysis/analysis_settings_NRR-W.yaml (task_id=c7d7f3e7-bb40-11f0-aa6a-027493648695)
2025-11-06 13:45:37.697 | INFO     | multiplex_pipeline.core_cutting.file_io:submit_all_transfers:91 - Submitted transfer for core_definitions to /ix1/kkedziora/NRR-W_Analysis/cores.pkl (task_id=c823f4d4-bb40-11f0-a2cb-027493648695)


In [9]:
# monitor transfer
print(f'Status of {transfer.pending[0][1]}: {tc.get_task(transfer.pending[0][0])["status"]}')
print(f'Status of {transfer.pending[1][1]}: {tc.get_task(transfer.pending[1][0])["status"]}')

Status of /ix1/kkedziora/NRR-W_Analysis/analysis_settings_NRR-W.yaml: SUCCEEDED
Status of /ix1/kkedziora/NRR-W_Analysis/cores.pkl: SUCCEEDED
